In [14]:
import pandas as pd
import numpy as np
import gurobipy as gp
from gurobipy import GRB

In [99]:
data1 = [[10e3, 15e3, 10e3, 20e3,  5e3],
         [10e3, 20e3, 15e3, 15e3, 15e3],
         [ 0,  0,  0,  0,  0]]

data2 = [[0.0, 0.0, 1.0e3, 1.5e3, 0.0],
         [0.0, 0.0, 1.4e3, 1.2e3, 0.0],
         [0.0, 0.0, 0.0, 0.0, 2.0e3],
         [0.0, 0.0, 0.0, 0.0, 0.7e3],
         [0.0, 0.0, 0.0, 0.0, 0.0]]

data3 = [[ 5, 14, 13],
         [14,  5,  9],
         [13,  9, 10]]

CITIES = ['Bristol', 'Brighton', 'London']
DEPTS = ['A', 'B', 'C', 'D', 'E']
DEPTS_COMB = {'A': ['B', 'C', 'D', 'E'],
              'B': ['C', 'D', 'E'],
              'C': ['D', 'E'],
              'D': ['E'],
              'E': []}

benefits = pd.DataFrame(data1, columns=DEPTS, index=CITIES)
comm_qty = pd.DataFrame(data2, columns=DEPTS, index=DEPTS).fillna(value=0)
comm_cost = pd.DataFrame(data3, columns=CITIES, index=CITIES)
comm_pairs = [(i, j, k, l) for i in DEPARTS for j in CITIES for k in DEPTS_COMB[i] for l in CITIES if comm_qty[k][i]>0]

In [144]:
model = gp.Model('Decentralisation')

# add vars
loc = model.addVars(DEPTS, CITIES,
                    name='loc',
                    vtype=GRB.BINARY)
comm = model.addVars(comm_pairs,
                     name='comm',
                    vtype=GRB.BINARY)

# obj function
model.setObjective((gp.quicksum(benefits[i][j]*loc[i,j] for i in DEPTS for j in CITIES) - 
                    gp.quicksum(comm_qty[k][i]*comm_cost[l][j]*comm[i,j,k,l] for i,j,k,l in comm_pairs)),
                   GRB.MAXIMIZE)

# constraints
model.addConstrs((gp.quicksum(loc[i,j] for j in CITIES) == 1 for i in DEPTS),
                  name='dept_in_city')
model.addConstrs((gp.quicksum(loc[i,j] for i in DEPTS) <= 3 for j in CITIES),
                 name='max_3_dept')
model.addConstrs((comm[i,j,k,l] - loc[i,j] <= 0 for i,j,k,l in comm_pairs),
                 name='if_comm_then_loc')
model.addConstrs((comm[i,j,k,l] - loc[k,l] <= 0 for i,j,k,l in comm_pairs),
                 name='if_comm_then_loc')
model.addConstrs((loc[i,j] + loc[k,l] - comm[i,j,k,l] <= 1 for i,j,k,l in comm_pairs),
                 name='if_loc_then_comm')

model.update()

In [145]:
model.write('8 Decentralisation.lp')
model.optimize()

Gurobi Optimizer version 9.1.0 build v9.1.0rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 170 rows, 69 columns and 408 nonzeros
Model fingerprint: 0x353be910
Variable types: 0 continuous, 69 integer (69 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [4e+03, 3e+04]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 3e+00]
Found heuristic solution: objective -54800.00000
Presolve time: 0.00s
Presolved: 170 rows, 69 columns, 408 nonzeros
Variable types: 0 continuous, 69 integer (69 binary)

Root relaxation: objective 6.750000e+04, 13 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 67500.0000    0   10 -54800.000 67500.0000   223%     -    0s
H    0     0                    -18000.00000 67500.0000   475%     -    0s
H    0     0             

In [148]:
print('Profit (benefits-costs) of ${:.2f}'.format(model.objVal))

for i, j in loc.keys():
    if loc[i,j].x >0:
        print('Department {} in {}'.format(i,j))

Profit (benefits-costs) of $14900.00
Department A in Bristol
Department B in Brighton
Department C in Brighton
Department D in Bristol
Department E in Brighton
